In [2]:
import torch
from torch import optim
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt
import numpy as np
import os

import laspy

In [50]:
class PointCloudDataset(Dataset):
    def __init__(self, pts_file):
        points = np.loadtxt(pts_file, delimiter=' ')
        # do i need to min max intensity, return number, number of returns, etc.? probably not
        for i in range(3):
            dim_min, dim_max = min(points[:,i]), max(points[:,i])
            points[:,i] = (points[:,i] - dim_min) / (dim_max - dim_min)
        self.data = points
        
    def __len__(self):
        return len(self.data) // 5000
        
    def __getitem__(self, idx):
        # Return batches of 5000 points
        xyzirn = self.data[idx * 5000: (idx + 1) * 5000, :6]  # x, y, z, intensity, return number, number of returns
        label = self.data[idx * 5000: (idx + 1) * 5000, 6] == 5

        xyzirn = torch.from_numpy(xyzirn.T).float()
        label = torch.tensor(label).long()
        
        return xyzirn, label

training_data = PointCloudDataset(r"C:\Users\and13375\Documents\3D Point Segmentation\Test\Vaihingen\3DLabeling\Vaihingen3D_Traininig.pts")
# training_data[0]

In [36]:
train_dataloader = DataLoader(training_data, batch_size=64, shuffle=False) # can/should i use shuffle

In [37]:
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

Feature batch shape: torch.Size([32, 6, 5000])
Labels batch shape: torch.Size([32, 5000])


In [45]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [46]:
class STNkd(nn.Module):
    def __init__(self, k=64):
        super(STNkd, self).__init__()
        self.mlp1 = nn.Sequential(torch.nn.Conv1d(k, 64, 1), nn.BatchNorm1d(64), nn.GELU())
        self.mlp2 = nn.Sequential(torch.nn.Conv1d(64, 128, 1), nn.BatchNorm1d(128), nn.GELU())
        self.mlp3 = nn.Sequential(torch.nn.Conv1d(128, 1024, 1), nn.BatchNorm1d(1024), nn.GELU())
        self.mlp4 = nn.Sequential(nn.Linear(1024, 512), nn.BatchNorm1d(512), nn.GELU())
        self.mlp5 = nn.Sequential(nn.Linear(512, 256), nn.BatchNorm1d(256), nn.GELU())
        self.fc = nn.Linear(256, k*k)

        self.k = k

    def forward(self, x):
        batchsize = x.size()[0]
        x = self.mlp1(x)
        x = self.mlp2(x)
        x = self.mlp3(x)

        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)

        x = self.mlp4(x)
        x = self.mlp5(x)
        x = self.fc(x)

        iden = Variable(torch.from_numpy(np.eye(self.k).flatten().astype(np.float32))).view(1,self.k*self.k).repeat(batchsize,1)
        if x.is_cuda:
            iden = iden.cuda()
        x = x + iden
        x = x.view(-1, self.k, self.k)
        return x

In [43]:
# Define model
class PoinNet(nn.Module):
    def __init__(self, input_dim = 3, k=10):
        super().__init__()
        # self.tnet1 = Tnet().to(device)
        # Should GELU be after LayerNorm?
        self.mlp1 = nn.Sequential(nn.Conv1d(6, 64, kernel_size=1), nn.BatchNorm1d(64), nn.GELU())
        self.mlp2 = nn.Sequential(nn.Conv1d(64, 64, kernel_size=1), nn.BatchNorm1d(64), nn.GELU())
        
        # self.tnet2 = Tnet().to(device)
        # can i call mlp2 again?
        self.mlp3 = nn.Sequential(nn.Conv1d(64, 64, kernel_size=1), nn.BatchNorm1d(64), nn.GELU())
        self.mlp4 = nn.Sequential(nn.Conv1d(64, 128, kernel_size=1), nn.BatchNorm1d(128), nn.GELU())
        self.mlp5 = nn.Sequential(nn.Conv1d(128, 1024, kernel_size=1), nn.BatchNorm1d(1024), nn.GELU())
        
        self.mlp6 = nn.Sequential(nn.Linear(1024, 512), nn.BatchNorm1d(512), nn.GELU())
        self.mlp7 = nn.Sequential(nn.Linear(512, 256), nn.BatchNorm1d(256), nn.GELU())
        self.fc = nn.Linear(256, k)

    def forward(self, x):
        x = self.mlp1(x)
        print(1, x.size())
        x = self.mlp2(x)
        print(2, x.size())
        
        x = self.mlp3(x)
        print(3, x.size())
        x = self.mlp4(x)
        print(4, x.size())
        x = self.mlp5(x)
        print(5, x.size())
        x = torch.max(x, 2)[0]
        print(6, x.size())
        
        x = self.mlp6(x)
        print(6, x.size())
        x = self.mlp7(x)
        print(7, x.size())
        x = self.fc(x)
        print(8, x.size())
        return F.log_softmax(x, dim=1)

model = PoinNet(input_dim = 6).to(device)

Using cuda device


In [44]:
out = model(train_features.to(device))
print("OUT", out.size())

1 torch.Size([32, 64, 5000])
2 torch.Size([32, 64, 5000])
3 torch.Size([32, 64, 5000])
4 torch.Size([32, 128, 5000])
5 torch.Size([32, 1024, 5000])
6 torch.Size([32, 1024])
6 torch.Size([32, 512])
7 torch.Size([32, 256])
8 torch.Size([32, 10])
OUT torch.Size([32, 10])


In [47]:
class SimplePointNet(nn.Module):
    def __init__(self, num_points = 500, num_classes = 2):
        super(SimplePointNet, self).__init__()
        self.num_points = num_points
        self.conv1 = nn.Conv1d(3, 64, 1)
        self.conv2 = nn.Conv1d(64, 128, 1)
        self.fc1 = nn.Linear(128, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 128)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# instantiate the model
model = SimplePointNet()

# to use the model, you'd pass in a batch of 3D points of shape (batch_size, 3, num_points)
# for example:
points = torch.rand(32, 3, 500)  # batch of 32, each with 500 points (x, y, z)
out = model(points)  # forward pass
out.size()

torch.Size([32, 2])

In [2]:
def train_PointNet(num_epochs, cnn, loaders, device):
    cnn.train()
    loss_func = nn.CrossEntropyLoss()
    optimizer = optim.Adam(cnn.parameters(), lr = 0.01)

    for epoch in range(num_epochs):
        for batch_idx, (images, labels) in enumerate(loaders['train']):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            out = cnn(images)
            loss = loss_func(out, labels)
            loss.backward()
            optimizer.step()
 
            print(f"Epoch: {epoch+1}\tBatch ID: {batch_idx}\tLoss: {loss.item()}")
            print()

train_PointNet(5, model, loaders, device)

NameError: name 'model' is not defined

In [16]:
def test_PointNet(cnn, loaders, device):
    cnn.eval()
    correct = 0
    total = 0
    with torch.no_grad():  # No need to track gradients in testing phase
        for images, labels in loaders['test']:
            images, labels = images.to(device), labels.to(device)
            outputs = cnn(images)
            _, predicted = torch.max(outputs.data, 1) # returns max values, index of max values
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy of the model on test images: {100 * correct / total}% on {total} values')
    
test_CNN(model, loaders, device)

1 torch.Size([64, 32, 24, 24])
2 torch.Size([64, 32, 12, 12])
3 torch.Size([64, 64, 8, 8])
4 torch.Size([64, 64, 4, 4])
5 torch.Size([64, 1024])
6 torch.Size([64, 256])
7 torch.Size([64, 10])
1 torch.Size([64, 32, 24, 24])
2 torch.Size([64, 32, 12, 12])
3 torch.Size([64, 64, 8, 8])
4 torch.Size([64, 64, 4, 4])
5 torch.Size([64, 1024])
6 torch.Size([64, 256])
7 torch.Size([64, 10])
1 torch.Size([64, 32, 24, 24])
2 torch.Size([64, 32, 12, 12])
3 torch.Size([64, 64, 8, 8])
4 torch.Size([64, 64, 4, 4])
5 torch.Size([64, 1024])
6 torch.Size([64, 256])
7 torch.Size([64, 10])
1 torch.Size([64, 32, 24, 24])
2 torch.Size([64, 32, 12, 12])
3 torch.Size([64, 64, 8, 8])
4 torch.Size([64, 64, 4, 4])
5 torch.Size([64, 1024])
6 torch.Size([64, 256])
7 torch.Size([64, 10])
1 torch.Size([64, 32, 24, 24])
2 torch.Size([64, 32, 12, 12])
3 torch.Size([64, 64, 8, 8])
4 torch.Size([64, 64, 4, 4])
5 torch.Size([64, 1024])
6 torch.Size([64, 256])
7 torch.Size([64, 10])
1 torch.Size([64, 32, 24, 24])
2 torch.S